In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('finance.db')
stock_list = pd.read_sql_query("SELECT * FROM stock_list;", conn)
stock_data = pd.read_sql_query("SELECT * FROM stock_data;", conn)
conn.close()

In [12]:
import pandas as pd

target_date = "2025-06-30"
price_threshold = 5 / 100
volume_threshold = 150 / 100

date_cols = [c for c in stock_data.columns if c not in ['Price','Ticker','시장','종목명']]

close_data = stock_data[stock_data['Price'] == 'Close'].copy()
close_only = close_data[date_cols].astype(float)

prev_date = pd.to_datetime(target_date) - pd.Timedelta(days=1)
prev_date = prev_date.strftime('%Y-%m-%d')
if prev_date not in close_only.columns:
    prev_date = close_only.columns[close_only.columns.get_loc(target_date) - 1]

close_data['현재종가'] = close_data[target_date]
close_data['이전종가'] = close_data[prev_date]
close_data['등락률'] = ((close_data['현재종가'] / close_data['이전종가']) - 1) * 100

volume_data = stock_data[stock_data['Price'] == 'Volume'].copy()
volume_only = volume_data[date_cols].astype(float)
if prev_date not in volume_only.columns:
    prev_date = volume_only.columns[volume_only.columns.get_loc(target_date) - 1]
volume_data['현재거래량'] = volume_data[target_date]
volume_data['이전거래량'] = volume_data[prev_date]
volume_data['거래량변화율'] = ((volume_data['현재거래량'] / volume_data['이전거래량']) - 1) * 100

merged = pd.merge(
    close_data[['Ticker','종목명','등락률']],
    volume_data[['Ticker','거래량변화율']],
    on='Ticker'
)

filtered = merged[(merged['등락률'] >= (price_threshold * 100)) & (merged['거래량변화율'] >= (volume_threshold * 100))]
result = filtered['종목명'].tolist()
print(result)

['휴스틸', '율촌화학', '동양철관', '태영건설우', '화천기계', '평화홀딩스', '삼화전자', '동원개발', '사조씨푸드', '신풍제약', '세원정공', '태광', '서울전자통신', '엔씨소프트', '파세코', '화성밸브', 'KD', '대동스틸', '대원미디어', '지어소프트', 'CJ프레시웨이', '메디콕스', '테이팩스', '시지메드텍', '세아홀딩스', '드림어스컴퍼니', '산일전기', '빛과전자', '하이스틸', '텔코웨어', '메디톡스', 'THE E&M', '넥스틸', '형지엘리트', 'SK이노베이션', 'SK이노베이션우', '한국화장품', '선진', '유티아이', '오늘이엔엠', '노바렉스', '미스터블루', 'FSN', '케어젠', '제테마', '티앤알바이오팹', '펌텍코리아', '실리콘투', '케일럼', '유틸렉스', '네오펙트', '압타바이오', '세아제강', '비투엔', '셀비온', '퀀타매트릭스', '프레스티지바이오로직스', '두산퓨얼셀2우B', '에이텀', '티와이홀딩스', '티와이홀딩스우', '지에프씨생명과학', '삐아', '에스엠씨지', '미트박스', '잉글우드랩', '프레스티지바이오파마']
